# Chapter 07

1️⃣ CNN 모델 구현하기

In [4]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

# 신경망 모델 구성
X = tf.placeholder(tf.float32, [None,28,28,1])   # None : 입력 데이터 개수, 1 : 특징 개수(회색 이미지라 채널이 1)
Y = tf.placeholder(tf.float32, [None,10])   # 출력값이 10개의 분류
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01))
L1 = tf.nn.conv2d(X,W1, strides = [1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides=[1,2,2,1], padding='SAME')  # 커널 크기를 2 * 2로 하는 풀링 계층

W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01))
L2 = tf.nn.conv2d(L1,W2, strides = [1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides = [1,2,2,1],padding='SAME')

W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev = 0.01))
L3 = tf.reshape(L2, [-1,7*7*64])
L3 = tf.matmul(L3,W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3,keep_prob)

W4 = tf.Variable(tf.random_normal([256,10], stddev = 0.01))
model = tf.matmul(L3,W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

# 신경망 모델 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs , batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1,28,28,1)
        
        _, cost_val = sess.run([optimizer,cost], feed_dict={X:batch_xs, Y: batch_ys, keep_prob:0.7})
        
        total_cost +=cost_val
        
    print('Epoch:', '%04d' %(epoch+1), 'Avg.cost = ', '{:.3f}'.format(total_cost/total_batch))
    
print('최적화 완료!')

# 결과 확인
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy, feed_dict={X:mnist.test.images.reshape(-1,28,28,1), Y:mnist.test.labels, keep_prob:1}))

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch: 0001 Avg.cost =  0.358
Epoch: 0002 Avg.cost =  0.113
Epoch: 0003 Avg.cost =  0.082
Epoch: 0004 Avg.cost =  0.063
Epoch: 0005 Avg.cost =  0.051
Epoch: 0006 Avg.cost =  0.043
Epoch: 0007 Avg.cost =  0.039
Epoch: 0008 Avg.cost =  0.035
Epoch: 0009 Avg.cost =  0.029
Epoch: 0010 Avg.cost =  0.028
Epoch: 0011 Avg.cost =  0.024
Epoch: 0012 Avg.cost =  0.022
Epoch: 0013 Avg.cost =  0.020
Epoch: 0014 Avg.cost =  0.018
Epoch: 0015 Avg.cost =  0.018
최적화 완료!
정확도: 0.9886


2️⃣ 고수준 API

In [9]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

# 신경망 모델 구성
X = tf.placeholder(tf.float32, [None,28,28,1])   # None : 입력 데이터 개수, 1 : 특징 개수(회색 이미지라 채널이 1)
Y = tf.placeholder(tf.float32, [None,10])   # 출력값이 10개의 분류
is_training = tf.placeholder(tf.bool)

L1 = tf.layers.conv2d(X,32,[3,3])
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2])
L1 = tf.layers.dropout(L1,0.7, is_training)

L2 = tf.layers.conv2d(L1,64,[3,3])
L2 = tf.layers.max_pooling2d(L2, [2,2], [2,2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3,256,activation=tf.nn.relu)
L3 = tf.layers.dropout(L3,0.5,is_training)

model = tf.layers.dense(L3,10, activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

# 신경망 모델 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs , batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1,28,28,1)
        
        _, cost_val = sess.run([optimizer,cost], feed_dict={X:batch_xs, Y: batch_ys, is_training:True})
        
        total_cost +=cost_val
        
    print('Epoch:', '%04d' %(epoch+1), 'Avg.cost = ', '{:.4f}'.format(total_cost/total_batch))
    
print('최적화 완료!')

# 결과 확인
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy, feed_dict={X:mnist.test.images.reshape(-1,28,28,1), Y:mnist.test.labels, is_training:False}))

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use keras.layers.Dense instead.
Epoch: 0001 Avg.cost =  0.1775
Epoch: 0002 Avg.cost =  0.0498
Epoch: 0003 Avg.cost =  0.0305
Epoch: 0004 Avg.cost =  0.0227
Epoch: 0005 Avg.cost =  0.0146
Epoch: 0006 Avg.cost =  0.0132
Epoch: 0007 Avg.cost =  0.0129
Epoch: 0008 Avg.cost =  0.0107
Epoch: 0009 Avg.cost =  0.0073
Epoch: 0010 Avg.cost =  0.0055
Epoch: 0011 Avg.cost =  0.0092
Epoch: 0012 Avg.cost =  0.0086
Epoch: 0013 Avg.cost =  0.0035
Epoch: 0014 Avg.cost =  0.0063
Epoch: 0015 Avg.cost =  0.0048
최적화 완료!
정확도: 0.9879
